In [ ]:
from IPython.display import display

import torch
from torch import nn
from torch.nn import functional

torch.set_printoptions(sci_mode=False)
torch.set_default_tensor_type('torch.FloatTensor')
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [ ]:
class Net (nn.Module):
    def __init__(self, n_channels):
        super(Net.self).__init__()
        
        self.n_channels = n_channels

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=self.n_channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(n_channels, n_channels*2, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(n_channels*2, n_channels*4, kernel_size=3, stride=1, padding=1)

        self.fc1 = nn.Linear(self.n_channels*4*8*8, self.n_channels*4) # Entrada la salida de n_dims de foward
        self.fc2 = nn.Linear(self.n_channels*4, 6)

    def foward(self, x):
        # Empieza 3x64x64
        x = self.conv1(x) # n_channels x 64 x 64
        x = functional.relu(functional.max_pool2d(x, 2)) # max_pool divide dimension de imagen | n_chanels x 32 x 32
        
        x = self.conv2(x) # n_channels*2 x 32 x 32
        x = functional.relu(functional.max_pool2d(x, 2)) # n_chanels*2 x 16 x 16
        
        x = self.conv3(x) # n_channels*4 x 16 x 16
        x = functional.relu(functional.max_pool2d(x, 2)) # n_chanels*4 x 8 x 8

        # Flatten, aplanar datos a una dimension
        x = x.view(-1, self.n_channels*4*8*8)
        
        # Fully Connected (FC)
        x = self.fc1(x) # out=n_cannels*4
        x = functional.relu(x) 
        x = self.fc2(x) # Salida cantidad de clases a predecir

        # log_SoftMax
        x = functional.log_softmax(x, dim=1) # Salida en forma de probabilidad

        return x